In [4]:
import cvxpy as cp, numpy as np

In [5]:
def makeA2(x,y):
    u = x.reshape((-1,1))
    v = y.reshape((-1,1))
    return (u*v.T+v*u.T) / 2.  # ip(u,v)

In [6]:
# Paramaters

N = 15     # no. iterations
L = 1     # h-smoothness constant
lam = 1/L # step-size
R = 1     # initial radius

In [7]:
dimG = 2*N + 3
dimF = dimH = N + 1
nbPts = N + 2

In [8]:
# encoding vectors 

x = np.zeros((nbPts, dimG))  # iterates x^*, x_0, x_1, \dots, x_N
df = np.zeros((nbPts, dimG)) # \nabla f
dh = np.zeros((nbPts, dimG)) # \nabla h
f = np.zeros((nbPts, dimF))  # function values
h = np.zeros((nbPts, dimH))  # kernel values

x[1:,:nbPts-1] = np.eye(nbPts-1)
df[1:,N+1:2*N+2] = np.eye(N+1)
dh[1,-1] = 1

f[1:,:nbPts-1] = np.eye(nbPts-1)
h = f.copy()

In [9]:
# encoding the NoLips algorithm

for i in range(N):
    dh[i+2,:] = dh[i+1,:] - lam * df[i+1,:]

In [10]:
G = cp.Variable((dimG, dimG), PSD=True)
F = cp.Variable((dimF, 1))
H = cp.Variable((dimH, 1))

# initial radius constraints
constraints = {'radius': (h[0,:] - h[1,:])@H - dh[1,:] @ G @ (x[0,:] - x[1,:]) <= R}

# convexity constraints
for i in range(nbPts):
    for j in range(nbPts):
        if i != j:
            # convexity of f
            constraints[f'f conv. ({i},{j})'] = (f[j,:] - f[i,:])@F + df[j,:] @ G @( x[i,:] - x[j,:]) <= 0
            
            # conveity of Lh - f
            constraints[f'Lh - f conv. ({i},{j})'] = L*(h[j,:] - h[i,:])@H - (f[j,:] - f[i,:])@F + (L*dh[j,:] - df[j,:]) @ G @ (x[i,:] - x[j,:]) <= 0

In [12]:
obj.value

0.06666666608618009

In [14]:
# objective
obj = cp.Maximize((f[-1,:] - f[0,:])@F)
prob = cp.Problem(obj,constraints.values())
prob.solve(solver=cp.MOSEK)
print('PEP value: {:.12f}'.format(obj.value))
print('Theo value: {:.12f}'.format(R/lam/N))

for name,c in constraints.items():
    if not np.isclose(c.dual_value, 0, atol=1e-4):
        print('\t{}: {:.12f}'.format(name, c.dual_value[0]))

PEP value: 0.066666666086
Theo value: 0.066666666667
	radius: 0.066666666120
	f conv. (0,1): 0.066666666401
	f conv. (0,2): 0.066666666441
	f conv. (0,3): 0.066666666490
	f conv. (0,4): 0.066666666507
	f conv. (0,5): 0.066666666517
	f conv. (0,6): 0.066666666602
	f conv. (0,7): 0.066666666627
	f conv. (0,8): 0.066666666654
	f conv. (0,9): 0.066666666700
	f conv. (0,10): 0.066666666657
	f conv. (0,11): 0.066666666737
	f conv. (0,12): 0.066666666734
	f conv. (0,13): 0.066666666759
	f conv. (0,14): 0.066666666780
	f conv. (0,15): 0.066666666802
	f conv. (0,16): 0.066666666814
	Lh - f conv. (0,16): 0.066666666788
	Lh - f conv. (2,1): 0.066666666051
	f conv. (2,3): 0.016566145727
	Lh - f conv. (2,3): 0.006292657633
	f conv. (2,4): 0.010273488103
	Lh - f conv. (2,4): 0.002453631702
	f conv. (2,5): 0.007819856405
	Lh - f conv. (2,5): 0.001479065577
	f conv. (2,6): 0.006340790827
	Lh - f conv. (2,6): 0.001093505461
	f conv. (2,7): 0.005247285363
	Lh - f conv. (2,7): 0.000880642800
	f conv. (2,